In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras import utils
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import cifar100

In [2]:
dataset1 = cifar10
(x_train1, y_train1), (x_test1, y_test1) = dataset1.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test1[0]        
#translate the data to image
x_train1 = x_train1.astype("float32")
x_test1 = x_test1.astype("float32")


#normalization
x_train1 /= 255
x_test1 /= 255

alextnet_10 = load_model('Model/alexnet_cifar10.h5')#load lenet5 model trained with mnist dataset 
alextnet_10.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [15]:
beta = 0.03998
N = 600
sample(x_test1[0],y_test1[0],alextnet_10,N,beta)

Sampled 818 images, 218 outside beta, 600 inside beta
600 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.03998


In [6]:
beta = 0.0345
N = 783
sample(x_test1[0],y_test1[0],alextnet_10,N, beta)

783 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0345


0.0345

In [8]:
beta = 0.0387
N = 671
sample(x_test1[0],y_test1[0],alextnet_10,N, beta)

671 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0387


0.0387

In [9]:
dataset2 = cifar100
(x_train2, y_train2), (x_test2, y_test2) = dataset2.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test2[0]        
#translate the data to image
x_train2 = x_train2.astype("float32")
x_test2 = x_test2.astype("float32")


#normalization
x_train2 /= 255
x_test2 /= 255

alextnet_100 = load_model('Model/alexnet_cifar100.h5')#load lenet5 model trained with mnist dataset 
alextnet_100.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [13]:
beta = 0.025
N = 600
sample(x_test2[0],y_test2[0],alextnet_100,N,beta)


Beta is reduced by 0.0001, the new beta is 0.024900000000000002
Beta is reduced by 0.0001, the new beta is 0.024800000000000003
Beta is reduced by 0.0001, the new beta is 0.024700000000000003
Beta is reduced by 0.0001, the new beta is 0.024600000000000004
Beta is reduced by 0.0001, the new beta is 0.024500000000000004
Beta is reduced by 0.0001, the new beta is 0.024400000000000005
Beta is reduced by 0.0001, the new beta is 0.024300000000000006
Beta is reduced by 0.0001, the new beta is 0.024200000000000006
Beta is reduced by 0.0001, the new beta is 0.024100000000000007
Beta is reduced by 0.0001, the new beta is 0.024000000000000007
Beta is reduced by 0.0001, the new beta is 0.023900000000000008
Beta is reduced by 0.0001, the new beta is 0.02380000000000001
Beta is reduced by 0.0001, the new beta is 0.02370000000000001
Beta is reduced by 0.0001, the new beta is 0.02360000000000001
Beta is reduced by 0.0001, the new beta is 0.02350000000000001
Beta is reduced by 0.0001, the new beta is 0

In [14]:
def sample(x_test,y_test,model,N, beta):
    
    max_distance = 0  #max safe distance
    count = 0
    count2 = 0
    neighbour_list=[]
    mean = np.zeros(3)  
    cov = np.eye(3)
    label = y_test
    
    while True:
        count2+=1
        p = np.random.multivariate_normal(mean, cov,(32,32)) #sample formula
        neighbour = p + x_test*255
        neighbour /= 255
        neighbour=neighbour.reshape(1,32,32,3) #reshape to the size that can be used in model
        distance=np.linalg.norm(p[0]) 
        distance /= 255
        #print(distance)
        if (distance < beta or distance == beta): #inside the radius
            
            output = model.predict(neighbour,batch_size=128)
            proba = softmax(output)
            pred = np.argmax(proba)
            
            if(pred == label): #correct prediction
                #neighbour_list.append((proba[label],neighbour))
                count+=1  
            else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                beta = beta-0.0001
                print("Beta is reduced by 0.0001, the new beta is",beta)   
            
        if(count==N):  
            #print("Sampled %s images, %s outside beta, %s inside beta" % (count2, count2-count,count))
            print("%s images inside beta have the same label, local robustness formula satisfied" %N)
            break

          
    
    print("max safe distance:",beta)            

In [4]:
def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

In [9]:
beta = 0.0162
N = 783
sample(x_test2[0],y_test2[0],alextnet_100,N, beta)

783 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0162


0.0162

In [10]:
beta = 0.0183
N = 671
sample(x_test2[0],y_test2[0],alextnet_100,N, beta)

671 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0183


0.0183